# Loading stuff

In [1]:
import pickle
import pandas as pd
import copy
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from util import *

with open('basket_data.pkl', 'rb') as file:
    save_dict = pickle.load(file)
    
players_df = pd.read_pickle('players_df')
games_df = pd.read_pickle('games_df')
winner_df = pd.read_pickle('winner_games.pkl')
winner_df = pd.DataFrame(winner_df)
org_games = save_dict['games']

In [5]:
players_df[players_df['game_id'] == '5202'][['Pts', 'team_name', 'team_id', 'player_id', 'player_name']]

,Pts,team_name,team_id,player_id,player_name
28056,0,rishon letsion (coach: avner yaor),253,8658,Assaf Oren
28057,16,rishon letsion (coach: avner yaor),253,8663,Dragan Stipanovic
28058,0,rishon letsion (coach: avner yaor),253,8666,Lior Lipshits
28059,0,rishon letsion (coach: avner yaor),253,8670,Omer Haimovic
28060,0,bnei herzelia (coach: erez bitman),421,8725,Gilad Breslaur
28061,24,Rishon Letsion,253,8673,Rowan Barrett
28062,18,B. Herzelia,421,8722,Andrew Kennedy
28063,24,B. Herzelia,421,8723,Bryan Bracey
28064,15,B. Herzelia,421,8726,Tyson Wheeler
28065,16,Rishon Letsion,253,8664,Tim Mason


# Optional: creating and saving history dictionary for each game

In [3]:
games_dict = {}
for index, game_line in tqdm(games_df.iterrows()):
    game_id = game_line['game_id']
    players_of_game = players_df[players_df['game_id'] == str(game_id)]
    games_hist_list = []
    for index, player_of_game in players_of_game.iterrows():
        player_hist_lines = create_history_obj(games_df, game_line, players_df, player_of_game['player_id'])
        games_hist_list.append(player_hist_lines)
    games_dict[str(game_id)] = games_hist_list

with open('games_dict_for_sol2.pkl','wb+') as file:
    pickle.dump(games_dict,file)

c:\python36\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\python36\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\python36\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\python36\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\python36\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\python36\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: set_value is deprecated 

KeyboardInterrupt: 

# Loading history dictionary for each game:

In [2]:
games_dict = pd.read_pickle('games_dict_for_sol2.pkl')

# Creating training matrix from games_dict

In [4]:
'''this cell computes the training matrix from the games dictionary. each row contains the data of a game. the second
dim of the matrix is the number of games backwards (the history). the third dim contains the stats of a player for each game.
the last dim is for the numer of players, which contains 26 players (maximum of 13 players for each team).
the rows are sorted by game date. the history for each player in a specific game includes all games he played up to this game
but not including the game itself. the results to predict (score delta) is saved in a matrix called total_delta_mat

data is saved in total_training_mat_for_neural.npy and total_delta_mat_for_neural.npy'''

col_num = 58 - 4 + 1  # removing stats of: datr, game_id, player_id, team_id plus is_player_exist
col_num = 58 - 3 + 1  # removing stats of: datr, game_id, team_id plus is_player_exist (if you want keeping player_id)
games_ids = list(games_dict.keys())
total_training_mat = np.zeros((len(games_ids), 85, col_num, 26)) # num_gamesX85(history_games)Xnum_statsXmax_num_players
total_delta_mat = np.zeros((len(games_ids), 1))
total_points_mat = np.zeros((len(games_ids), 26))
# assuming that for each game in games_df we know that the player_is_home is valid
count = 0
emtpy_idxs = []
problem = []
for game_id in tqdm(games_ids):
    total_dates_for_game = []
    one_train_line = np.zeros((85, col_num, 26))
    players_list = games_dict[game_id]
    for one_player_games in players_list:
        dates = one_player_games['datr'].tolist()
        total_dates_for_game += dates
    total_dates_for_game = np.sort(np.unique(total_dates_for_game))[::-1]
    cur_game = games_df[games_df['game_id'].astype(str) == (game_id)]
    assert cur_game['datr'].tolist()[0] == total_dates_for_game[0]
    home_ids = []
    away_ids = []
    for k in range(13):
        home_ids.append(cur_game['player_home' + str(k)].tolist()[0])
        away_ids.append(cur_game['player_away' + str(k)].tolist()[0])

    total_ids = np.sort(home_ids + away_ids)
    total_ids = total_ids[total_ids > 0]
    from_game_ids = np.sort(players_df[players_df['game_id'] == str(game_id)]['player_id'].tolist()).astype(int)
    if not np.all(total_ids == from_game_ids):
        problem.append(count)
    #total_dates_for_game = total_dates_for_game[1:] #DELETING THE GAME THAT WE WANT TO PREDICT
    home_count = 0
    away_count = 0
    date_count = 0
    for date in total_dates_for_game:
        home_count = 0
        away_count = 0
        home_already_inside = []
        away_already_inside = []

        for one_player_games in players_list:
            player_id = one_player_games['player_id'].tolist()[0]
            selected_game = one_player_games[one_player_games['datr'] == date]
            if len(selected_game) > 0:
                selected_game = selected_game[selected_game.index == selected_game.index[0]] #for some reason we sometimes have 2 selected games with same game_id
            if player_id == '0':
                continue
            if int(player_id) in home_ids and not (int(player_id) in home_already_inside):
                index = home_count
                home_count += 1
                home_already_inside.append(int(player_id))
            elif int(player_id) in home_ids and int(player_id) in home_already_inside:
                #one_train_line[date_count, :, index] = np.zeros((col_num))
                continue
            elif int(player_id) in away_ids and not (int(player_id) in away_already_inside):
                index = away_count + 13
                away_count += 1
                away_already_inside.append(int(player_id))
            else:
                #one_train_line[date_count, :, index] = np.zeros((col_num))
                continue

            if len(selected_game) == 0:
                one_train_line[date_count, :, index] = np.zeros((col_num))
            else:
                selected_game_for_train = copy.deepcopy(selected_game)
                selected_game_for_train = selected_game_for_train.drop('datr', axis=1)
                #selected_game_for_train = selected_game_for_train.drop('player_id', axis=1) #for new training we need the matrix to contain the player's id  
                selected_game_for_train = selected_game_for_train.drop('team_id', axis=1)
                selected_game_for_train = selected_game_for_train.drop('game_id', axis=1)
                selected_game_for_train['is_player_exist'] = 1
                one_train_line[date_count, :, index] = selected_game_for_train.as_matrix().astype(float)

        date_count += 1
    if np.all(one_train_line == 0):
        emtpy_idxs.append(count)
    total_training_mat[count, :, :, :] = one_train_line
    total_delta_mat[count, 0] = (cur_game['home_team_points'] - cur_game['away_team_points']).tolist()[0]
    count += 1

c:\python36\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [1]:
np.save('total_training_mat_sol2', total_training_mat)
np.save('total_delta_mat_sol2', total_delta_mat)

NameError: name 'np' is not defined